In [2]:
import numpy as np
import torch
import os

In [5]:
for model in os.listdir('/dccstor/hoo-misha-1/wilds/wilds/pretrained/camelyon17'):
    model = model[11:-4]
    save_path = f'/dccstor/hoo-misha-1/wilds/wilds/pretrained/camelyon17/camelyon17_{model}.pth'
    d = torch.load(save_path, map_location=torch.device('cpu'))
    alg = d['algorithm']
    if 'model.classifier.weight' in alg:
        weight = alg['model.classifier.weight']
        bias = alg['model.classifier.bias']
    else:
        weight = alg['classifier.weight']
        bias = alg['classifier.bias']
    np.save(f'/dccstor/hoo-misha-1/wilds/WOODS/camelyon17_{model}_classifier_weight.npy', weight)
    np.save(f'/dccstor/hoo-misha-1/wilds/WOODS/camelyon17_{model}_classifier_bias.npy', bias)
